In [3]:
import pandas as pd
from mainnet_launch.constants import (
    CACHE_TIME,
    eth_client,
    ALL_AUTOPOOLS,
    BAL_ETH,
    AUTO_ETH,
    AUTO_LRT,
    AutopoolConstants,
)
from mainnet_launch.solver_diagnostics.solver_diagnostics import _load_solver_df


solver_df = _load_solver_df(AUTO_ETH)
solver_df

,timestamp,sodOnly,chainId,solverAddress,poolAddress,destinationOut,tokenOut,amountOut,amountOutETH,destinationIn,tokenIn,minAmountIn,minAmountInETH,steps,sod,rebalanceTest,addRank,date,moveName
12,1726517778,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100024362286265712640,99858676116613316608,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726517778, 'destStates':...","{'currentTimestamp': 1726517778, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 2.32566398...",2024-09-16 20:16:18,autoETH idle -> toke-WETH-pxethweth
16,1726539377,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100025090337777090560,99858938038984015872,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726539377, 'destStates':...","{'currentTimestamp': 1726539377, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.14394204...",2024-09-17 02:16:17,autoETH idle -> toke-WETH-pxethweth
0,1726540837,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100017120806606127104,99850988038171295744,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726540837, 'destStates':...","{'currentTimestamp': 1726540837, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.17297496...",2024-09-17 02:40:37,autoETH idle -> toke-WETH-pxethweth
47,1726540976,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100017120806606127104,99850988038171295744,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726540976, 'destStates':...","{'currentTimestamp': 1726540976, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.17297496...",2024-09-17 02:42:56,autoETH idle -> toke-WETH-pxethweth
15,1726602027,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,99961014198875766784,99976617022487003136,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726602027, 'destStates':...","{'currentTimestamp': 1726602027, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.93728886...",2024-09-17 19:40:27,autoETH idle -> toke-WETH-pxethweth
25,1726632022,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,314500371054658584576,314500371054658584576,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,314380623530208002048,314405825418452926464,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726632022, 'destStates':...","{'currentTimestamp': 1726632022, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 4.35440614...",2024-09-18 04:00:22,autoETH idle -> toke-WETH-pxethweth
1,1726676459,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,146034134985157378048,146034134985157378048,toke-WETH-pxsteth,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,144676018833887887360,145995352037732876288,"[{'stepType': 'swap', 'de

In [44]:
all_steps = solver_df["steps"].values
dex_steps = []
for steps in all_steps:
    for step in steps:
        if "dex" in step:
            dex_steps.append(step)
pd.DataFrame.from_records(dex_steps)["dex"].value_counts()

dex
0x      49
lifi    22
Name: count, dtype: int64

In [55]:
def compute_number_of_add_rank_def(row: dict):
    addRank = row["addRank"]


solver_df

,timestamp,sodOnly,chainId,solverAddress,poolAddress,destinationOut,tokenOut,amountOut,amountOutETH,destinationIn,tokenIn,minAmountIn,minAmountInETH,steps,sod,rebalanceTest,addRank,date,moveName
12,1726517778,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100024362286265712640,99858676116613316608,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726517778, 'destStates':...","{'currentTimestamp': 1726517778, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 2.32566398...",2024-09-16 20:16:18,autoETH idle -> toke-WETH-pxethweth
16,1726539377,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100025090337777090560,99858938038984015872,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726539377, 'destStates':...","{'currentTimestamp': 1726539377, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.14394204...",2024-09-17 02:16:17,autoETH idle -> toke-WETH-pxethweth
0,1726540837,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100017120806606127104,99850988038171295744,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726540837, 'destStates':...","{'currentTimestamp': 1726540837, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.17297496...",2024-09-17 02:40:37,autoETH idle -> toke-WETH-pxethweth
47,1726540976,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,100017120806606127104,99850988038171295744,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726540976, 'destStates':...","{'currentTimestamp': 1726540976, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.17297496...",2024-09-17 02:42:56,autoETH idle -> toke-WETH-pxethweth
15,1726602027,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000000,100000000000000000000,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,99961014198875766784,99976617022487003136,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726602027, 'destStates':...","{'currentTimestamp': 1726602027, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 3.93728886...",2024-09-17 19:40:27,autoETH idle -> toke-WETH-pxethweth
25,1726632022,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,314500371054658584576,314500371054658584576,toke-WETH-pxethweth,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,314380623530208002048,314405825418452926464,"[{'stepType': 'swap', 'dex': '0x', 'tokenOut':...","{'currentTimestamp': 1726632022, 'destStates':...","{'currentTimestamp': 1726632022, 'type': 'From...","[[Tokemak-Wrapped Ether-pxETH/wETH, 4.35440614...",2024-09-18 04:00:22,autoETH idle -> toke-WETH-pxethweth
1,1726676459,False,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH idle,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,146034134985157378048,146034134985157378048,toke-WETH-pxsteth,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,144676018833887887360,145995352037732876288,"[{'stepType': 'swap', 'de

In [54]:
solver_df["addRank"].values[0]
# 1st no. should be the predicted ETH gain net of swap costs, 2nd no. is the swap costs. the results are highly dependent on the swap path that aggregators find

[['Tokemak-Wrapped Ether-pxETH/wETH',
  2.325663981257803e+18,
  1.633843689704325e+17],
 ['Tokemak-Wrapped Ether-Curve.fi Factory Pool: ETHx-ETH',
  2.0422473620885652e+18,
  4.505492978813174e+17],
 ['Tokemak-Wrapped Ether-Balancer ETHx/wstETH',
  1.4894010516595108e+18,
  3.4796047902398874e+17],
 ['Tokemak-Wrapped Ether-Gyroscope ECLP wstETH/wETH',
  1.2699583156727457e+18,
  1.567067505383506e+17],
 ['Tokemak-Wrapped Ether-Balancer rETH Stable Pool',
  1.0257218404206694e+18,
  1.451961140245627e+17],
 ['Tokemak-Wrapped Ether-osETH/rETH',
  9.767300862717005e+17,
  3.826184933475287e+17],
 ['Tokemak-Wrapped Ether-Balancer osETH/wETH StablePool',
  6.351343844973682e+17,
  2.2212840343764992e+17],
 ['Tokemak-Wrapped Ether-Balancer wstETH-WETH Stable Pool',
  5.879901311080856e+17,
  1.3756874073197773e+17],
 ['Tokemak-Wrapped Ether-Balancer pxETH/wETH StablePool',
  5.869533509021196e+17,
  1.849910908379136e+17],
 ['Tokemak-Wrapped Ether-wETH/rETH',
  3.718605018301774e+17,
  1.62

In [57]:
solver_df["moveName"][0]

'autoETH idle -> toke-WETH-pxethweth'

In [7]:
# Ensure that the 'date' column is in datetime format
solver_df["date"] = pd.to_datetime(solver_df["date"])

# Group by date and check if there is at least one row for each day
days_with_data = solver_df.groupby(solver_df["date"].dt.date).size() > 0

# Convert it to a DataFrame or Series to display the result
days_with_data_df = pd.DataFrame(days_with_data, columns=["Solver Ran At Least"])
days_with_data_df

,Has_data
date,
2024-09-16,True
2024-09-17,True
2024-09-18,True
2024-09-19,True
2024-09-20,True
2024-09-21,True
2024-09-22,True
2024-09-23,True
2024-09-24,True


In [14]:
import plotly.express as px

In [5]:
solver_df["date"]

12   2024-09-16 20:16:18
16   2024-09-17 02:16:17
0    2024-09-17 02:40:37
47   2024-09-17 02:42:56
15   2024-09-17 19:40:27
25   2024-09-18 04:00:22
1    2024-09-18 16:20:59
11   2024-09-19 02:56:00
9    2024-09-19 13:24:21
50   2024-09-19 21:58:04
36   2024-09-20 06:01:22
2    2024-09-20 20:38:59
22   2024-09-21 04:53:00
30   2024-09-21 14:53:20
40   2024-09-21 23:28:03
4    2024-09-22 08:00:15
31   2024-09-22 17:21:45
33   2024-09-23 02:21:03
38   2024-09-23 12:56:49
28   2024-09-23 22:00:50
21   2024-09-24 06:13:08
6    2024-09-24 15:54:40
46   2024-09-24 17:29:38
49   2024-09-24 18:01:24
26   2024-09-24 23:24:42
5    2024-09-25 04:33:40
35   2024-09-25 14:25:15
41   2024-09-25 22:46:36
14   2024-09-26 15:13:20
37   2024-09-27 03:11:47
42   2024-09-27 15:41:30
43   2024-09-28 02:21:51
39   2024-09-28 17:05:14
13   2024-09-29 02:48:26
34   2024-09-29 15:22:12
44   2024-09-30 00:18:55
3    2024-09-30 13:19:26
7    2024-09-30 22:30:21
19   2024-10-01 07:30:21
27   2024-10-01 16:30:21


In [2]:
contract = eth_client.eth.contract(BAL_ETH.autopool_eth_addr, abi=ERC_20_ABI)
events = get_each_event_in_contract(contract)

In [3]:
transfers = events["Transfer"]
transfers

,from,to,value,event,block,transaction_index,log_index,hash
0,0x0000000000000000000000000000000000000000,0x000000000000000000000000000000000000dEaD,100000,Transfer,20722909,21,171,0xb254f901f1c3a9eae581bc03bfd4043da6b3b9d7c8b7...
1,0x0000000000000000000000000000000000000000,0x123cC4AFA59160C6328C0152cf333343F510e5A3,100000000000000000,Transfer,20764874,19,160,0xd3dd60de82836115608e9a5f7b4980722efb3495bad8...
2,0x0000000000000000000000000000000000000000,0xC45e939ca8C43822A2A233404Ecf420712084c30,200000000000000000,Transfer,20764889,122,370,0x42b46f163cb6a252ec73dd5f0d8cef308a2d80a6de9a...
3,0xC45e939ca8C43822A2A233404Ecf420712084c30,0x74c260AE655DeddF3F5705d2498A726DCA2512c1,200000000000000000,Transfer,20764889,122,376,0x42b46f163cb6a252ec73dd5f0d8cef308a2d80a6de9a...
4,0x0000000000000000000000000000000000000000,0x9DbC5E35a2f1A6dC214Bdb7ed7a8c706D77f0A74,10000000000000000000,Transfer,20764916,37,225,0xe4ebfeec1742a5d0ba84ee48b92f7d29f6dfd2e734a2...
...,...,...,...,...,...,...,...,...
459,0x0d949de5277Ebd781D66F4644e784d1d583a9a2B,0xC45e939ca8C43822A2A233404Ecf420712084c30,17513840536583446053,Transfer,20820316,56,353,0x2ef1fe7bffb416b649fe17916266669f09558d3de2c8...
460,0xC45e939ca8C43822A2A233404Ecf420712084c30,0x74c260AE655DeddF3F5705d2498A726DCA2512c1,17513840536583446053,Transfer,20820316,56,356,0x2ef1fe7bffb416b649fe17916266669f09558d3de2c8...
461,0x0000000000000000000000000000000000000000,0x4C0169B48c5A22503F1C3B871b921d55024A5939,65242835274913373,Transfer,20820577,166,563,0x83f4aa1972ab5c96b1abd5dd033bc0644bdb8946405c...
462,0x0000000000000000000000000000000000000000,0x4C0169B48c5A22503F1C3B871b921d55024A5939,112131699093657785,Transfer,20820577,166,565,0x83f4aa1972ab5c96b1abd5dd033bc0644bdb8946405c...


In [4]:
transfers = transfers.sort_values("block", ascending=False)

In [8]:
def build_wallet_balance_at_the_end_of_each_block(token_address: str) -> pd.DataFrame:
    # returns a dataframe of the balance of each token at the end of each block
    # breaks if a token_address is rebasing, or if balances can change while not emitting a transfer event

    contract = eth_client.eth.contract(token_address, abi=ERC_20_ABI)
    transfers = fetch_events(contract.events.Transfer)
    all_wallets = set(transfers["to"].unique())
    blocks = transfers["block"].unique().astype(int)
    block_and_timestamp_df = get_raw_state_by_blocks([], blocks, include_block_number=True).reset_index()

    def build_state_change(row: dict):
        amount = float(int(row["value"]) / 1e18)
        changes = {
            row["to"]: amount,
            row["from"]: -amount,
            "block": row["block"],
        }

        for w in all_wallets:
            if w not in changes:
                changes[w] = 0.0
        return changes

    rolling_wallet_erc20_balance_by_block = pd.DataFrame.from_records(
        transfers.sort_values("block", ascending=False).apply(build_state_change, axis=1)
    )
    rolling_wallet_erc20_balance_by_block = (
        rolling_wallet_erc20_balance_by_block.groupby("block").sum().cumsum().reset_index()
    )
    rolling_wallet_erc20_balance_by_block = pd.merge(
        rolling_wallet_erc20_balance_by_block, block_and_timestamp_df, on="block", how="left"
    )
    rolling_wallet_erc20_balance_by_block.set_index("timestamp", inplace=True)
    return rolling_wallet_erc20_balance_by_block


rolling_wallet_erc20_balance_by_block = build_wallet_balance_at_the_end_of_each_block(AUTO_LRT.autopool_eth_addr)
rolling_wallet_erc20_balance_by_block

,block,0x9597C17f323160D8462c194D72FAAcF7b4CBCd2b,0xC45e939ca8C43822A2A233404Ecf420712084c30,0x8b4334d4812C530574Bd4F2763FcD22dE94A969B,0x9DbC5E35a2f1A6dC214Bdb7ed7a8c706D77f0A74,0xf9F061d9a5f04E223ed3c560E03A8AB0A70Cc1c4,0x1415D6586E2dD9Cd5cfE3ff0C086c26356DB888e,0x64B3F224e41C6A4D8409eE544c633d467D05211F,0xe4c66916d44dCa977231eae4Eb9d34154ae39E7b,0xbc4Cf0A3A38328bf1C3B65BefA8C5665D620A0f4,...,0x2C21aB32B410B45f9eDfB3978Fa71F2aeB8F8e24,0x0c30eD59A7d409E5F43fF46f744Edd93f9E8E7e7,0xd8A9C0F3A6A09A625869Cea15aE3171B42E14bC0,0x003F35595dce3187B4Fff2B5A2c4303f7158208a,0x60A9EfDc632dF2c72Ca88dEDB29B4B0457089db5,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x1F378305a40cD8b5b5289a372F9A529B23eA1875,0x9Bc513230b112490527EB6d07594aE9473C4c1EE,0xF68D2BfCecd7895BBa05a7451Dd09A1749026454,0x0000000000000000000000000000000000000000
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-10 21:31:11,20722910,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000e+00,0.0,0.00,0.0,-1.000000e-13
2024-09-13 16:12:23,20742806,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000e+00,0.0,0.02,0.0,-2.000000e-02
2024-09-13 16:17:59,20742834,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000e+00,0.0,0.02,0.0,-1.200000e-01
2024-09-13 16:19:23,20742841,0.001000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000e+00,0.0,0.02,0.0,-1.210000e-01
2024-09-13 16:20:35,20742846,0.101000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000e+00,0.0,0.02,0.0,-1.210000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24 05:04:35,20818241,1267.366044,-3.608225e-16,440.082329,0.0,0.0,0.200104,0.0,0.0,0.0,...,0.0,0.0,0.001,0.0,0.0,-5.551115e-17,0.0,0.00,0.0,-1.708121e+03
2024-09-24 05:05:35,20818246,1267.566148,-3.608225e-16,440.082329,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.001,0.0,0.0,-5.551115e-17,0.0,0.00,0.0,-1.708121e+03
2024-09-24 12:54:23,20820586,1267.566148,-3.608225e-16,440.082329,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.001,0.0,0.0,2.771101e-01,0.0,0.00,0.0,-1.708541e+03


In [19]:
addresses_to_exclude = [
    "0x0000000000000000000000000000000000000000",
    *[a.autopool_eth_addr for a in ALL_AUTOPOOLS],
    "0x000000000000000000000000000000000000dEaD",
    "0x9597C17f323160D8462c194D72FAAcF7b4CBCd2b",  # main rewarder
    "0xC45e939ca8C43822A2A233404Ecf420712084c30",  # router
]

cols = [c for c in rolling_wallet_erc20_balance_by_block.columns if c not in ["block", *addresses_to_exclude]]

In [28]:
def _daily_balance_over_zero(group):

    for value in group:
        if value > 0.0:
            return True

    return False


balance_lists = rolling_wallet_erc20_balance_by_block[cols].resample("1D").apply(list).map(_daily_balance_over_zero)

balance_lists.sum(axis=1)

timestamp
2024-09-10     0
2024-09-11     0
2024-09-12     0
2024-09-13     4
2024-09-14     2
2024-09-15     4
2024-09-16    13
2024-09-17    12
2024-09-18     5
2024-09-19     9
2024-09-20    12
2024-09-21     9
2024-09-22     7
2024-09-23     5
2024-09-24     7
Freq: D, dtype: int64

In [14]:
rolling_wallet_erc20_balance_by_block.resample("1D").apply(list)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [ ]:
# had some balance within the last d ay

In [11]:
rolling_wallet_erc20_balance_by_block.columns.sort_values()

Index(['0x0000000000000000000000000000000000000000',
       '0x000000000000000000000000000000000000dEaD',
       '0x003F35595dce3187B4Fff2B5A2c4303f7158208a',
       '0x0397dd68931395E5594BBeA98B2F46dfcf5A4243',
       '0x09618943342c016A85aC0F98Fd005479b3cec571',
       '0x0c30eD59A7d409E5F43fF46f744Edd93f9E8E7e7',
       '0x123cC4AFA59160C6328C0152cf333343F510e5A3',
       '0x1395f53B0472827c78964316a7f5595f302e10c9',
       '0x13cCDb2080483d7cf9545f496457d393994B7Da6',
       '0x1415D6586E2dD9Cd5cfE3ff0C086c26356DB888e',
       '0x1F378305a40cD8b5b5289a372F9A529B23eA1875',
       '0x2326D4fb2737666DDA96bd6314e3D4418246cFE8',
       '0x2C21aB32B410B45f9eDfB3978Fa71F2aeB8F8e24',
       '0x41c8B18F310a03D95cC1Bbf879E08A021Fd6cf67',
       '0x47B2FAABf5DF79E92C0B95D7064749741AaaCc87',
       '0x4C0169B48c5A22503F1C3B871b921d55024A5939',
       '0x5013725853c272A08Ca424736436DCc041BAe29F',
       '0x60A9EfDc632dF2c72Ca88dEDB29B4B0457089db5',
       '0x64B3F224e41C6A4D8409eE544c633d467D05

In [12]:
def get_count_of_balance_above_zero(row:dict):
    print(row.keys())

    
    addresses
    for addr in row:
        if addr not in :
            pass
            
    
    return   

rolling_wallet_erc20_balance_by_block.apply(get_count_of_balance_above_zero, axis=1)

Index(['block', '0x9597C17f323160D8462c194D72FAAcF7b4CBCd2b',
       '0xC45e939ca8C43822A2A233404Ecf420712084c30',
       '0x8b4334d4812C530574Bd4F2763FcD22dE94A969B',
       '0x9DbC5E35a2f1A6dC214Bdb7ed7a8c706D77f0A74',
       '0xf9F061d9a5f04E223ed3c560E03A8AB0A70Cc1c4',
       '0x1415D6586E2dD9Cd5cfE3ff0C086c26356DB888e',
       '0x64B3F224e41C6A4D8409eE544c633d467D05211F',
       '0xe4c66916d44dCa977231eae4Eb9d34154ae39E7b',
       '0xbc4Cf0A3A38328bf1C3B65BefA8C5665D620A0f4',
       '0x09618943342c016A85aC0F98Fd005479b3cec571',
       '0x000000000000000000000000000000000000dEaD',
       '0xFa4FC4ec2F81A4897743C5b4f45907c02ce06199',
       '0x925dB2228A00f4bC0Fb627618e71542ECdd24B17',
       '0x0397dd68931395E5594BBeA98B2F46dfcf5A4243',
       '0x5013725853c272A08Ca424736436DCc041BAe29F',
       '0x2326D4fb2737666DDA96bd6314e3D4418246cFE8',
       '0xEDae60F1d7837B13EDa3D3bA9F4dA6b8463c07c8',
       '0xA68b37C561D656a41E05874551979fE1C794206F',
       '0xD18aB6A2aADA43486b9DFF4f892

timestamp
2024-09-10 21:31:11    None
2024-09-13 16:12:23    None
2024-09-13 16:17:59    None
2024-09-13 16:19:23    None
2024-09-13 16:20:35    None
                       ... 
2024-09-24 05:04:35    None
2024-09-24 05:05:35    None
2024-09-24 12:54:23    None
2024-09-24 14:30:11    None
2024-09-24 14:30:59    None
Length: 284, dtype: object

In [19]:
dfs = [_fetch_raw_deposit_and_withdrawal_dfs(a) for a in ALL_AUTOPOOLS]
dfs

2024-09-23 20:58:57.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:58:57.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

[(                                                         sender  \
  timestamp                                                         
  2024-09-16 18:10:35  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-16 18:13:35  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-16 18:18:59  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-16 18:19:47  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-16 18:21:23  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  ...                                                         ...   
  2024-09-23 19:37:59  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-23 19:59:59  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-23 21:05:23  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-23 21:10:59  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  2024-09-23 22:18:23  0xC45e939ca8C43822A2A233404Ecf420712084c30   
  
                                                            owner  \
  timestamp                    

In [20]:
d, w = dfs[0]
d["sender"].value_counts()

sender
0xC45e939ca8C43822A2A233404Ecf420712084c30    151
0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5     21
Name: count, dtype: int64

In [21]:
d["owner"].value_counts()

owner
0xC45e939ca8C43822A2A233404Ecf420712084c30    79
0x4C0169B48c5A22503F1C3B871b921d55024A5939    21
0x123cC4AFA59160C6328C0152cf333343F510e5A3     6
0x5013725853c272A08Ca424736436DCc041BAe29F     6
0x09618943342c016A85aC0F98Fd005479b3cec571     4
0x99AfD53f807766A8B98400B0C785E500c041F32B     2
0x14c6150db396FF57007f01E4b560D16C72905e51     2
0x77438638274A752823D330189ceE4b64d51BB93D     2
0x0c98361F38Ed48A5a287Cc8e4d07f22E5FcbcaCF     2
0x2cD7Ca2EE4ef60bf2d725B82572c659F6f2cf3E2     2
0x8b4334d4812C530574Bd4F2763FcD22dE94A969B     2
0x87313004cd08D0B342B97d6C494497B51FE79b0c     1
0x30f29Ca88311F4cc1A1314bc1c45752982D7FD67     1
0xab557f77Ef6d758A18DF60AcfaCB1d5feE4C09c2     1
0x9DbC5E35a2f1A6dC214Bdb7ed7a8c706D77f0A74     1
0x7D424157d3c5ae94f6adB98BBfeDA2Ed48d70d8b     1
0xFe9D9B55F80C23436c36DFbe7F6Fc8dC3BD1F587     1
0xcc2Cc53bAA6BFfbFeA9012855A50b39640fAC575     1
0x7c2eA10D3e5922ba3bBBafa39Dc0677353D2AF17     1
0x0E1145C24A90b8542E882Fe8b6a75a5FA06B38A0     1
0xD29adE875357